In [ ]:
!pip install vitaldb matplotlib awscli pypdf --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.2.1 re

In [ ]:
import vitaldb
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

def plot_and_save_patient_data(case_id: int, output_dir: str, pdf_pages: PdfPages):
    """
    Downloads, plots, and saves vital signs data for a single patient to a PDF page.

    Args:
        case_id: The ID of the patient case (e.g., 1 for 0001.vital).
        output_dir: The directory where temporary files might be stored.
        pdf_pages: A PdfPages object to which the generated figure will be added.
    """
    # Format case_id to match filename convention (e.g., 1 -> 0001)
    case_filename = f'{case_id:04d}.vital'
    file_path = os.path.join(output_dir, case_filename)
    s3_path = f's3://physionet-open/vitaldb/1.0.0/vital_files/{case_filename}'

    print(f"Processing Case {case_id}...")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Download the vital file
    download_command = f'aws s3 cp --no-sign-request {s3_path} {file_path}'
    os.system(download_command)

    if not os.path.exists(file_path):
        print(f"ERROR: Download failed for '{case_filename}'. File not found.")
        return

    try:
        vf = vitaldb.VitalFile(file_path)

        # Define tracks to load - reverted to the original set for the 3-subplot view
        tracks_to_load = [
            'Solar8000/HR',
            'Solar8000/ART_MBP',
            'Solar8000/PLETH_SPO2',
            'Solar8000/ETCO2',
            'BIS/BIS',
            'EVENT'
        ]

        df = vf.to_pandas(tracks_to_load, interval=1)

        # Determine the plotting time range
        start_time, end_time = df.index.min(), df.index.max()
        if 'Solar8000/ART_MBP' in df.columns:
            valid_start = df['Solar8000/ART_MBP'].first_valid_index()
            valid_end = df['Solar8000/ART_MBP'].last_valid_index()
            if valid_start is not None and valid_end is not None:
                start_time = valid_start
                end_time = valid_end
        # Add padding to the time range
        start_time = max(df.index.min(), start_time - 300)
        end_time = min(df.index.max(), end_time + 300)


        events = df['EVENT'].dropna()

        # Create the figure and 3 subplots, sharing the x-axis
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 15), sharex=True) # Reverted to 3 subplots
        fig.suptitle(f'Comprehensive Patient View (Case {case_id:04d})', fontsize=18) # Dynamic title

        # --- Subplot 1: Core Vitals (HR, MAP) ---
        ax1.plot(df.index, df['Solar8000/HR'], label='Heart Rate (bpm)', color='dodgerblue', marker='.', markersize=2, linestyle='-')
        ax1.plot(df.index, df['Solar8000/ART_MBP'], label='Mean Arterial Pressure (mmHg)', color='orangered', marker='.', markersize=2, linestyle='-')
        ax1.set_ylabel('Core Vitals')
        ax1.legend(loc='upper left')
        ax1.grid(True, linestyle='--', alpha=0.6)
        ax1.set_ylim(0, 150)

        # --- Subplot 2: Oxygenation and Ventilation (SpO2, EtCO2) ---
        ax2.plot(df.index, df['Solar8000/PLETH_SPO2'], label='SpO2 (%)', color='darkcyan', marker='.', markersize=2, linestyle='-')
        ax2.set_ylabel('SpO2 (%)', color='darkcyan')
        ax2.tick_params(axis='y', labelcolor='darkcyan')
        ax2.set_ylim(85, 101)

        ax2_twin = ax2.twinx()
        ax2_twin.plot(df.index, df['Solar8000/ETCO2'], label='EtCO2 (mmHg)', color='gold', marker='.', markersize=2, linestyle='-')
        ax2_twin.set_ylabel('EtCO2 (mmHg)', color='gold') # Reverted label
        ax2_twin.tick_params(axis='y', labelcolor='gold')
        ax2_twin.set_ylim(0, 60)

        # Combine legends for ax2 and ax2_twin
        lines, labels = ax2.get_legend_handles_labels()
        lines2, labels2 = ax2_twin.get_legend_handles_labels()
        ax2_twin.legend(lines + lines2, labels + labels2, loc='upper left')
        ax2.grid(True, linestyle='--', alpha=0.6)

        # --- Subplot 3: Anesthetic Depth (BIS) ---
        ax3.plot(df.index, df['BIS/BIS'], label='BIS Score', color='green')
        ax3.axhspan(40, 60, color='green', alpha=0.1, label='Target Anesthesia Range')
        ax3.set_ylabel('Anesthetic Depth (BIS)')
        ax3.legend(loc='upper left')
        ax3.grid(True, linestyle='--', alpha=0.6)
        ax3.set_ylim(0, 100)

        # --- No Subplot 4 for Anesthetic Agents ---

        # --- Add Event Markers to all subplots ---
        for time, text in events.items():
            for ax in [ax1, ax2, ax3]: # Apply to 3 subplots only
                ax.axvline(x=time, color='red', linestyle='--', linewidth=1.5)
            ax1.text(time + 30, 140, text, rotation=90, verticalalignment='top', color='red', fontsize=10)

        ax1.set_xlim(start_time, end_time)

        plt.tight_layout(rect=[0, 0.03, 1, 0.97])
        pdf_pages.savefig(fig)
        plt.close(fig)
        print(f"Successfully plotted and added Case {case_id} to PDF.")

    except Exception as e:
        print(f"Error processing Case {case_id}: {e}")
    finally:
        # Clean up the downloaded vital file
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Cleaned up '{case_filename}'.")


# --- Main execution for batch processing ---
NUM_PATIENTS = 100 # Set to 100 for full run
OUTPUT_DIR = 'patient_plots'
COMBINED_PDF_FILENAME = 'all_patient_vitals_3_subplots.pdf' # Changed filename to reflect 3 subplots

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Create a PdfPages object to save all plots into a single PDF
combined_pdf_path = os.path.join(OUTPUT_DIR, COMBINED_PDF_FILENAME)
with PdfPages(combined_pdf_path) as pdf:
    for case_id in range(1, NUM_PATIENTS + 1):
        plot_and_save_patient_data(case_id, OUTPUT_DIR, pdf)

print(f"\nAll plots combined into: {combined_pdf_path}")
print(f"You can find individual vital files downloaded and removed in {OUTPUT_DIR} during processing.")

Processing Case 1...
Successfully plotted and added Case 1 to PDF.
Cleaned up '0001.vital'.
Processing Case 2...
Successfully plotted and added Case 2 to PDF.
Cleaned up '0002.vital'.
Processing Case 3...
Successfully plotted and added Case 3 to PDF.
Cleaned up '0003.vital'.
Processing Case 4...
Successfully plotted and added Case 4 to PDF.
Cleaned up '0004.vital'.
Processing Case 5...
Successfully plotted and added Case 5 to PDF.
Cleaned up '0005.vital'.
Processing Case 6...
Successfully plotted and added Case 6 to PDF.
Cleaned up '0006.vital'.
Processing Case 7...
Successfully plotted and added Case 7 to PDF.
Cleaned up '0007.vital'.
Processing Case 8...
Successfully plotted and added Case 8 to PDF.
Cleaned up '0008.vital'.
Processing Case 9...
Successfully plotted and added Case 9 to PDF.
Cleaned up '0009.vital'.
Processing Case 10...
Successfully plotted and added Case 10 to PDF.
Cleaned up '0010.vital'.
Processing Case 11...
Successfully plotted and added Case 11 to PDF.
Cleaned u